In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

In [2]:
def getHTMLText(url):
    try:
        headers = {'user-agent':'Mozilla/5.0'}
        r = requests.get(url,timeout=30,headers=headers)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return "产生异常"

In [3]:
def getPageurl(list,pageNum):
    first_url = "http://search.cnki.com.cn/Search.aspx?q=%e5%8c%ba%e5%9d%97%e9%93%be&rank=relevant&cluster=all&val=&p="
    for i in range(pageNum):
        i = i*15
        soup = BeautifulSoup(getHTMLText(first_url+str(i)),'html.parser')
        for div in soup.find_all('div',class_="wz_tab"):
            for a in div.find_all('a',target='_blank'):
                if 'http://search.cnki.net' not in a.get('href'):
                    list.append(a.get('href'))

In [4]:
def getOnePage(list,pagetext):
    for oneurl in list:
        try:
            onepage={}
            tempL=[]
            soup = BeautifulSoup(getHTMLText(oneurl),'html.parser')
            onepage['题目']=soup.head.title.string#题目
            onepage['关键词'] = soup.head.find_all('meta')[3].get('content')#关键词
            au = []
            for div in soup.find_all('div',style="text-align:center; width:740px; height:30px;"):#作者
                for a in div.find_all('a',target="_blank"):
                    au.append(a.string)
            onepage['作者'] = au
            for div in soup.find_all('div',style='float:left;'):
                for b in div.find_all('b'):#机构
                    onepage['机构']=b.string.strip()
                for font in div.find_all('font',color='#0080ff'):#年份
                    onepage['年份']=font.string.strip()
            for div in soup.find_all('div',id="div_Ref"):#相似文献、引用文献等
                ref=[]
                for td in div.find_all('td',rowspan="2",align="left",valign="bottom",class_="b14"):
                    getType = td.string[1:5]
                for a in div.find_all('a',target="_blank"):
                    ref.append(a.string)
                onepage[getType] = ref
            for table in soup.find_all('table',cellspacing="0",cellpadding="0",width="100%",style="border:1px solid #7498d6;"):
                temp=[]
                for a in table.find_all('a',target="_blank"):#相关机构和相关作者
                    temp.append(a.string)
                tempL.append(temp)
            if len(tempL) > 0:
                onepage['相关机构'] = tempL[len(tempL)-2]
                onepage['相关作者'] = tempL[len(tempL)-1]
            pagetext.append(onepage)
        except:
            pass
        continue

In [18]:
ls = []
pagetext=[]
getPageurl(ls,20)#设置爬取页数
getOnePage(ls,pagetext) 

In [19]:
frame = pd.DataFrame(pagetext)

In [20]:
frame.head() 

""


In [8]:
writer = pd.ExcelWriter('G:/CDO/知网研学/CNKI-analysis-master/Jupyter notebook/1.xlsx')

In [9]:
frame.to_excel(writer,'Sheet1')

In [10]:
writer.save()